In [1]:
import tensorflow as tf
import numpy as np
import os
import time
from PIL import Image
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import shutil

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
#shutil.rmtree("/home/ubuntu/wgan/model/")
os.mkdir("/home/ubuntu/wgan/model/")

In [4]:
class WGAN(object):
    def __init__(self, input_width, input_height, output_width, output_height, z_dim, d_iters=5, g_iters=1):
        self.z_dim = z_dim
        self.d_iters = d_iters
        self.g_iters = g_iters
        self.input_width = input_width
        self.input_height = input_height
        self.output_width = output_width
        self.output_height = output_height


    def build_model(self):
        with tf.name_scope("input"):
            self.z = tf.placeholder(tf.float32, [None, self.z_dim], name='z')
            self.imgs = tf.placeholder(tf.float32, [None, self.input_height, self.input_width, 3], name='real_images')

        with tf.name_scope("generator_output"):
            self.g_img = self.generator(self.z)

        with tf.name_scope("discriminate_loss"):
            d_real = self.discriminator(self.imgs)
            d_fake = self.discriminator(self.g_img)
           # Value Funciton, Discriminator最大化Value Function
            self.d_loss = tf.reduce_mean(d_real) - tf.reduce_mean(d_fake)


        with tf.name_scope("generator_loss"):
           # WGAN Generator 的损失函数
            self.g_loss = -tf.reduce_mean(d_fake)


        with tf.name_scope("optimizer_op"):
            self.d_trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "discriminator")
            self.g_trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "generator")
            # WGAN 推荐采用 RMSPropOptimizer
            self.d_optimizer = tf.train.RMSPropOptimizer(learning_rate=5e-5).minimize(-self.d_loss,
                                                  var_list=self.d_trainable_vars)
            # 对Discriminator weight clipping 近似满足1−𝐿𝑖𝑝𝑠𝑐ℎ𝑖𝑡𝑧
            self.clip_d = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in self.d_trainable_vars]
            self.g_optimizer = tf.train.RMSPropOptimizer(learning_rate=8e-5).minimize(
                self.g_loss, var_list=self.g_trainable_vars)

    def train(self, sess, batch_imgs, batch_z): 
        loss_d = 0
        loss_g = 0
        # 论文推荐Discriminator的更新轮数要大于Generator, 确保Discriminator 衡量真实分布与生成分布的差异更为准确
        # 论文的更新次数为5
        for i in range(self.d_iters):
            loss_d, _, _ = sess.run([self.d_loss, self.d_optimizer, self.clip_d], feed_dict={self.imgs: batch_imgs, self.z:batch_z})
        # 训练Generator 论文的更新次数为1
        for i in range(self.g_iters):
            loss_g, _ = sess.run([self.g_loss, self.g_optimizer], feed_dict={self.imgs: batch_imgs, self.z: batch_z})
        return -loss_d, loss_g

    def discriminator(self, img):
        """
        区别于对图片进行分类(只需要抽象信息), discriminator的目的是判定图片的“真实性”
        因此Discriminator不采用pooling 这样有损图片结构的操作
        """
        with tf.variable_scope("discriminator", reuse=tf.AUTO_REUSE):
            d_layer_1 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(tf.contrib.layers.conv2d(img, num_outputs=64, activation_fn=None, kernel_size=[5, 5], stride=2, reuse=tf.AUTO_REUSE , weights_initializer=tf.random_normal_initializer(stddev=0.02), padding='SAME', scope='d_layer_1')))
            d_layer_2 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(tf.contrib.layers.conv2d(d_layer_1, num_outputs=128, activation_fn=None, kernel_size=[5, 5], stride=2, reuse=tf.AUTO_REUSE, weights_initializer=tf.random_normal_initializer(stddev=0.02), padding='SAME', scope='d_layer_2')))
            d_layer_3 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(tf.contrib.layers.conv2d(d_layer_2, num_outputs=256, activation_fn=None, kernel_size=[5, 5], stride=2, reuse=tf.AUTO_REUSE, weights_initializer=tf.random_normal_initializer(stddev=0.02), padding='SAME', scope='d_layer_3')))
            d_layer_4 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(tf.contrib.layers.conv2d(d_layer_3,num_outputs=512,  activation_fn=None, kernel_size=[5, 5], stride=2, reuse=tf.AUTO_REUSE, weights_initializer=tf.random_normal_initializer(stddev=0.02), padding='SAME', scope='d_layer_4')))
            flatten = tf.layers.flatten(d_layer_4)
            logits = tf.layers.dense(flatten, 1, reuse=tf.AUTO_REUSE, name='d_layer_5', activation=None)
        return logits

    def generator(self, z):
        """
        generator的目标是生成图片，如何从给定的向量去生成图片？
        采用反卷积，将图片不断增大
        """
        with tf.variable_scope("generator", reuse=False):
            g_layer_1 = tf.nn.relu(tf.contrib.layers.batch_norm(tf.layers.dense(z, (self.input_width // 16) * (self.output_height // 16) * 512, activation=None, reuse=False)))
            g_layer_1 = tf.reshape(g_layer_1, [-1, self.input_width // 16, self.output_height // 16, 512])
            g_parameters = ( 
                            {"num_outputs": 256, "kernel_size": [3, 3], "stride": 2},
                            {"num_outputs": 128, "kernel_size": [3, 3], "stride": 2},
                            {"num_outputs": 64, "kernel_size": [3, 3], "stride": 2},
                            )
            x = g_layer_1
            for parameters in g_parameters:
                x = tf.nn.relu(tf.contrib.layers.batch_norm(tf.contrib.layers.conv2d_transpose(x, **parameters, activation_fn=None, reuse=False, padding='SAME', weights_initializer=tf.random_normal_initializer(stddev=0.02))))
            img = tf.contrib.layers.conv2d_transpose(x, num_outputs=3, kernel_size=[3, 3], activation_fn=tf.nn.tanh, stride=2, reuse=False, padding='SAME', weights_initializer=tf.random_normal_initializer(stddev=0.02))
        return img

    def infer(self, sess, model_dir, icon_dir, z_dim):
        z_sample = np.random.normal(0, 1, [25, z_dim]).astype(np.float32)
        #with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        ckpt = tf.train.get_checkpoint_state(model_dir)
        print(ckpt)
        self.saver = tf.train.Saver(max_to_keep=10)
        if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
            print('Reloading model parameters...')
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(sess=sess, save_path=os.path.join(model_dir, ckpt_name))
        samples = sess.run(self.g_img, feed_dict={self.z: z_sample})
        save_images(samples, icon_dir)

    def visualize_result(self, samples):
        fig = plt.figure(figsize=(5, 5))
        gs = gridspec.GridSpec(5, 5)
        gs.update(wspace=0.1, hspace=0.1)
        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample)
        return fig

def save_images(images, image_path):
    gen_imgs = inverse_transform(images)
    r, c = 5, 5
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(image_path + "icon1.png")
    plt.close()

def inverse_transform(images):
    #return  (images + 1.) / 2.
    return np.array(((images+1.)/2.)*255., dtype=int)

def get_batch(path, batch_size, z_dim):
    files = os.listdir(path)
    np.random.shuffle(files)
    batch_imgs = []
    for file in files:
        img = np.array(Image.open(path + file).resize((64, 64)))
        img = (img / 255.) * 2. - 1
        batch_imgs.append(img)
        if len(batch_imgs) == batch_size:
            batch_z = np.random.normal(0, 1, [batch_size, z_dim]).astype(np.float32)
            yield batch_imgs, batch_z
            batch_imgs = []
    if len(batch_imgs) != 0:
        batch_z = np.random.normal(0, 1, [len(batch_imgs), z_dim]).astype(np.float32)
        yield batch_imgs, batch_z

In [ ]:
img_path = "/home/ubuntu/wgan/faces/"
path = "/home/ubuntu/wgan"
pic_path = "/home/ubuntu/wgan"
model_dir = "/home/ubuntu/wgan/model/"
train_batch_size = 128
epochs = 102
z_dim = 100
g = tf.Graph()
gpu_options = tf.GPUOptions(allow_growth=True)
d_loss = 0
g_loss = 0
with g.as_default():
    wgan = WGAN(input_height=64, input_width=64, output_height=64, output_width=64, z_dim=z_dim)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
        wgan.build_model()
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        saver = tf.train.Saver(max_to_keep=10)
        ckpt = tf.train.get_checkpoint_state(model_dir)
        # 读取model的存档,如果有存档，从存档点开始接着训练
        if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
            print('Reloading model parameters...')
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            saver.restore(sess=sess, save_path=os.path.join(model_dir, ckpt_name))
        for epoch_id in range(epochs):
            start = time.time()
            d_loss = 0
            g_loss = 0
            train_data = get_batch(path=img_path, batch_size=train_batch_size, z_dim=z_dim)
            for batch_imgs, batch_z in train_data:
                d_loss_batch, g_loss_batch = wgan.train(sess, batch_imgs, batch_z)
                d_loss += d_loss_batch
                g_loss += g_loss_batch
            end = time.time()
            print("Epoch %d / %d, " % (epoch_id + 1, epochs),
                  "Discriminator Train Loss: %.4f" % d_loss,
                  "Generator Train Loss: %.4f" % g_loss,
                  "%.4f sec / epoch" % ((end - start)))
            saver.save(sess, model_dir, global_step=epoch_id)
        wgan.infer(sess, model_dir=model_dir, icon_dir=pic_path, z_dim=z_dim)






